# Riparian ecosystem Classificattion

In [8]:
import cv2
import numpy as np
import os

# Set the directory containing the image dataset
dataset_dir = 'Dataset/riparian/'

# Set the threshold for vegetation segmentation
lower_green = np.array([40, 40, 40])
upper_green = np.array([70, 255, 255])

# Apply a median filter to reduce noise
median_kernel_size = 5

# Apply an opening morphological operation to remove small or isolated areas
opening_kernel_size = 5

# Create the "Processed" directory if it doesn't exist
if not os.path.exists('Output/Riparian'):
    os.makedirs('Output/Riparian')
if not os.path.exists('Output/Riparian/HSV'):
    os.makedirs('Output/Riparian/HSV')
if not os.path.exists('Output/Riparian/Morphological'):
    os.makedirs('Output/Riparian/Morphological')
if not os.path.exists('Output/Riparian/Contour'):
    os.makedirs('Output/Riparian/Contour')
if not os.path.exists('Output/Riparian/Final_Output'):
    os.makedirs('Output/Riparian/Final_Output')

# Iterate over the images in the dataset directory
f. ilename in os.listdir(dataset_dir):
    # Load the image
    image = cv2.imread(os.path.join(dataset_dir, filename))

    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Save the HSV image
    cv2.imwrite(os.path.join('Output/Riparian/HSV', f"{filename.split('.')[0]}_hsv.jpg"), hsv)

    # Apply the threshold to the HSV image to create a binary mask for vegetation
    mask_veg = cv2.inRange(hsv, lower_green, upper_green)

    # Apply a median filter to the binary mask to reduce noise
    mask_veg = cv2.medianBlur(mask_veg, median_kernel_size)

    # Apply an opening morphological operation to the binary mask to remove small or isolated areas
    kernel = np.ones((opening_kernel_size, opening_kernel_size), np.uint8)
    mask_veg = cv2.morphologyEx(mask_veg, cv2.MORPH_OPEN, kernel)
       
    # Save the processed binary mask image after morphological operation
    cv2.imwrite(os.path.join('Output/Riparian/Morphological', f"{filename.split('.')[0]}_binary_mask.jpg"), mask_veg)

    # Convert the binary mask to a color image for drawing contours
    mask_veg_gray = cv2.cvtColor(mask_veg, cv2.COLOR_GRAY2BGR)

    # Find the contours of the vegetation mask
    contours_veg, _ = cv2.findContours(mask_veg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours of the vegetation zone on the original image
    layer_veg = np.zeros_like(image)
    cv2.drawContours(layer_veg, contours_veg, -1, (0, 255, 0), 3)
    
    # Save the processed image after contouring the vegetation zone
    cv2.imwrite(os.path.join('Output/Riparian/Contour', f"{filename.split('.')[0]}_contoured.jpg"), layer_veg)

    # Draw the contours of the vegetation zone boundaries on the original image
    layer_veg_bound = np.zeros_like(image)
    total_area = 0  # variable to accumulate the areas of all contours in the image
    total_perimeter = 0 # variable to accumulate the perimeters of all contours in the imag
    for contour in contours_veg:

        # Calculate the area of the contour and add it to the total area
        area = cv2.contourArea(contour)
        total_area += area
        perimeter = cv2.arcLength(contour, True)
        total_perimeter += perimeter
        
        # Draw the contour and perimeter on the image
        cv2.drawContours(layer_veg_bound, contour, -1, (0, 255, 0), 3)
        cv2.putText(layer_veg_bound, f"{perimeter:.2f}", tuple(contour[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Add the area text to the image
    area_text = f"Vegetation area: {total_area:.2f} sq. pixels"
    perimeter_text = f"Vegetation perimeter: {total_perimeter:.2f} pixels"
    cv2.putText(layer_veg_bound, area_text, (10, image.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
    cv2.putText(layer_veg_bound, perimeter_text, (10, image.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
    
    # Create a new row for the CSV file with the filename, total area, and total perimeter
    row = [filename, total_area, total_perimeter]

   
    # Print the total vegetation area to the console
    print(f"{filename}: Vegetation area = {total_area:.2f} sq. pixels, Vegetation perimeter = {total_perimeter:.2f} pixels")

    # Save the processed image
    cv2.imwrite(os.path.join('Output/Riparian/Final_Output', f"{filename.split('.')[0]}_processed.jpg"), layer_veg_bound)
    


LC08_L1TP_147048_20140131_20200912_02_T1.jpg: Vegetation area = 201532.50 sq. pixels, Vegetation perimeter = 29984.01 pixels
LC08_L1TP_147048_20150203_20200909_02_T1.jpg: Vegetation area = 173149.00 sq. pixels, Vegetation perimeter = 26796.90 pixels
LC08_L1TP_147048_20160206_20200907_02_T1.jpg: Vegetation area = 150078.50 sq. pixels, Vegetation perimeter = 25532.41 pixels
LC08_L1TP_147048_20170208_20200905_02_T1.jpg: Vegetation area = 164911.50 sq. pixels, Vegetation perimeter = 26784.61 pixels
LC08_L1TP_147048_20180211_20200902_02_T1.jpg: Vegetation area = 145998.00 sq. pixels, Vegetation perimeter = 24200.95 pixels
LC08_L1TP_147048_20190214_20200829_02_T1.jpg: Vegetation area = 133833.00 sq. pixels, Vegetation perimeter = 22879.41 pixels
LC08_L1TP_147048_20200201_20200823_02_T1.jpg: Vegetation area = 152577.00 sq. pixels, Vegetation perimeter = 26163.81 pixels
LC08_L1TP_147048_20210203_20210303_02_T1.jpg: Vegetation area = 164505.50 sq. pixels, Vegetation perimeter = 28152.82 pixels


# Forest ecosystem Classificattion

In [9]:
import cv2
import numpy as np
import os

# Set the directory containing the image dataset
dataset_dir = 'Dataset/Forest'

# Set the threshold for vegetation segmentation
lower_green = np.array([25, 50, 20])
upper_green = np.array([90, 255, 255])

# Apply a median filter to reduce noise
median_kernel_size = 5

# Apply an opening morphological operation to remove small or isolated areas
opening_kernel_size = 5

# Create the "Processed" directory if it doesn't exist
if not os.path.exists('Output/Forest'):
    os.makedirs('Output/Forest')
if not os.path.exists('Output/Forest/HSV'):
    os.makedirs('Output/Forest/HSV')
if not os.path.exists('Output/Forest/Morphological'):
    os.makedirs('Output/Forest/Morphological')
if not os.path.exists('Output/Forest/Contour'):
    os.makedirs('Output/Forest/Contour')
if not os.path.exists('Output/Forest/Final_Output'):
    os.makedirs('Output/Forest/Final_Output')

# Iterate over the images in the dataset directory
for filename in os.listdir(dataset_dir):
    # Load the image
    image = cv2.imread(os.path.join(dataset_dir, filename))

    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
     # Save the HSV image
    cv2.imwrite(os.path.join('Output/Forest/HSV', f"{filename.split('.')[0]}_hsv.jpg"), hsv)

    # Apply the threshold to the HSV image to create a binary mask for vegetation
    mask_veg = cv2.inRange(hsv, lower_green, upper_green)

    # Apply a median filter to the binary mask to reduce noise
    mask_veg = cv2.medianBlur(mask_veg, median_kernel_size)

    # Apply an opening morphological operation to the binary mask to remove small or isolated areas
    kernel = np.ones((opening_kernel_size, opening_kernel_size), np.uint8)
    mask_veg = cv2.morphologyEx(mask_veg, cv2.MORPH_OPEN, kernel)
    
     # Save the processed binary mask image after morphological operation
    cv2.imwrite(os.path.join('Output/Forest/Morphological', f"{filename.split('.')[0]}_binary_mask.jpg"), mask_veg)

    # Convert the binary mask to a color image for drawing contours
    mask_veg_gray = cv2.cvtColor(mask_veg, cv2.COLOR_GRAY2BGR)

    # Find the contours of the vegetation mask
    contours_veg, _ = cv2.findContours(mask_veg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours of the vegetation zone on the original image
    layer_veg = np.zeros_like(image)
    cv2.drawContours(layer_veg, contours_veg, -1, (0, 255, 0), 3)
    
    # Save the processed image after contouring the vegetation zone
    cv2.imwrite(os.path.join('Output/Forest/Contour', f"{filename.split('.')[0]}_contoured.jpg"), layer_veg)

    # Draw the contours of the vegetation zone boundaries on the original image
    layer_veg_bound = np.zeros_like(image)
    total_area = 0  # variable to accumulate the areas of all contours in the image
    total_perimeter = 0 # variable to accumulate the perimeters of all contours in the imag
    for contour in contours_veg:
        cv2.drawContours(layer_veg_bound, contour, -1, (0, 255, 0), 3)

        # Calculate the area of the contour and add it to the total area
        area = cv2.contourArea(contour)
        total_area += area
        perimeter = cv2.arcLength(contour, True)
        total_perimeter += perimeter

        # Draw the contour and perimeter on the image
        cv2.drawContours(layer_veg_bound, contour, -1, (0, 255, 0), 3)
        cv2.putText(layer_veg_bound, f"{perimeter:.2f}", tuple(contour[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)        
    
    # Add the area text to the image
    area_text = f"Vegetation area: {total_area:.2f} sq. pixels"
    perimeter_text = f"Vegetation perimeter: {total_perimeter:.2f} pixels"
    cv2.putText(layer_veg_bound, area_text, (10, image.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
    cv2.putText(layer_veg_bound, perimeter_text, (10, image.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
    
    # Print the total vegetation area to the console
    print(f"{filename}: Vegetation area = {total_area:.2f} sq. pixels,Vegetation perimeter = {total_perimeter:.2f} pixels")

    # Save the processed image
    cv2.imwrite(os.path.join('Output/Forest/Final_Output', f"{filename.split('.')[0]}_processed.jpg"), layer_veg_bound)


LC08_L1TP_144046_20140211_20200911_02_T1.jpg: Vegetation area = 763685.50 sq. pixels,Vegetation perimeter = 34665.03 pixels
LC08_L1TP_144046_20150214_20200910_02_T1.jpg: Vegetation area = 774264.00 sq. pixels,Vegetation perimeter = 42805.55 pixels
LC08_L1TP_144046_20160217_20200907_02_T1.jpg: Vegetation area = 728104.00 sq. pixels,Vegetation perimeter = 43220.15 pixels
LC08_L1TP_144046_20170219_20200905_02_T1.jpg: Vegetation area = 797427.50 sq. pixels,Vegetation perimeter = 26909.54 pixels
LC08_L1TP_144046_20180222_20200902_02_T1.jpg: Vegetation area = 507974.50 sq. pixels,Vegetation perimeter = 42228.36 pixels
LC08_L1TP_144046_20190209_20200829_02_T1.jpg: Vegetation area = 709725.50 sq. pixels,Vegetation perimeter = 40079.79 pixels
LC08_L1TP_144046_20200212_20200823_02_T1.jpg: Vegetation area = 779496.50 sq. pixels,Vegetation perimeter = 24862.09 pixels
LC08_L1TP_144046_20210214_20210301_02_T1.jpg: Vegetation area = 770798.50 sq. pixels,Vegetation perimeter = 35424.25 pixels
LC09_L1T

In [2]:
import cv2
import os

image_dir = "Dataset/riparian/"

for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path)
    height, width, channels = img.shape
    area = height * width
    print("Image: {}, Area: {}  sq. pixels".format(filename, area))

Image: LC08_L1TP_147048_20140131_20200912_02_T1.jpg, Area: 1069056  sq. pixels
Image: LC08_L1TP_147048_20150203_20200909_02_T1.jpg, Area: 1069056  sq. pixels
Image: LC08_L1TP_147048_20160206_20200907_02_T1.jpg, Area: 1069056  sq. pixels
Image: LC08_L1TP_147048_20170208_20200905_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_147048_20180211_20200902_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_147048_20190214_20200829_02_T1.jpg, Area: 1069056  sq. pixels
Image: LC08_L1TP_147048_20200201_20200823_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_147048_20210203_20210303_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_147048_20220206_20220212_02_T1.jpg, Area: 1069056  sq. pixels
Image: LC08_L1TP_147048_20230209_20230217_02_T1.jpg, Area: 1069056  sq. pixels


In [3]:
import cv2
import os

image_dir = "Dataset/Forest"

for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path)
    height, width, channels = img.shape
    area = height * width
    print("Image: {}, Area: {}  sq. pixels".format(filename, area))

Image: LC08_L1TP_144046_20140211_20200911_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20150214_20200910_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20160217_20200907_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20170219_20200905_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20180222_20200902_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20190209_20200829_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20200212_20200823_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC08_L1TP_144046_20210214_20210301_02_T1.jpg, Area: 1070080  sq. pixels
Image: LC09_L1TP_144046_20220225_20220225_02_T1.jpg, Area: 1068032  sq. pixels
Image: LC09_L1TP_144046_20230212_20230308_02_T1.jpg, Area: 1068032  sq. pixels
